<a href="https://colab.research.google.com/github/Jack3690/Rad/blob/master/TensorFlow_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%tensorflow_version 1.x

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import math

In [16]:
def create_placeholders(n_x, n_y):

    X = tf.placeholder(tf.float32,(n_x,None),name='X')
    Y = tf.placeholder(tf.float32,(n_y,None),name='Y')
    
    return X, Y

In [44]:
def initialize_parameters(Layer_dims):
    tf.set_random_seed(1)                   # so that your "random" numbers match ours

    parameters={}
    L=len(Layer_dims)  

    for l in range(1,L):
        parameters['W'+str(l)] = tf.get_variable("W"+str(l), [Layer_dims[l],Layer_dims[l-1]], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        parameters['b'+str(l)] = tf.get_variable("b" +str(l), [Layer_dims[l],1], initializer = tf.zeros_initializer())
   
    return parameters

In [18]:
parameters

{'W1': <tf.Variable 'W1:0' shape=(25, 12288) dtype=float32_ref>,
 'W2': <tf.Variable 'W2:0' shape=(12, 25) dtype=float32_ref>,
 'W3': <tf.Variable 'W3:0' shape=(2, 12) dtype=float32_ref>,
 'b1': <tf.Variable 'b1:0' shape=(25, 1) dtype=float32_ref>,
 'b2': <tf.Variable 'b2:0' shape=(12, 1) dtype=float32_ref>,
 'b3': <tf.Variable 'b3:0' shape=(2, 1) dtype=float32_ref>}

In [19]:
def convert_to_one_hot(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(C,name='C')
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(labels, C,axis=0)
    
    # Create the session (approx. 1 line)
    sess = tf.Session()
    
    # Run the session (approx. 1 line)
    one_hot = sess.run(one_hot_matrix )
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    ### END CODE HERE ###
    
    return one_hot.squeeze()

In [49]:
def forward_propagation(X, parameters):
    
    # Retrieve the parameters from the dictionary "parameters" 

    L=len(parameters)//2
    A=X
    for l in range(1,L+1):
      Z= tf.matmul(parameters["W"+ str(l)],A) + parameters["b"+ str(l)]
      A= tf.nn.relu(Z)  
    ZL=Z
    return ZL

In [21]:
def compute_cost(ZL, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(ZL)
    labels = tf.transpose(Y)
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    ### END CODE HERE ###
    
    return cost

In [22]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [46]:
def model(X_train, Y_train, X_test, Y_test,Layer_dims,learning_rate = 0.0001, num_epochs = 1500, minibatch_size = 32, print_cost = True):
 

    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters(Layer_dims)
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    ZL = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(ZL, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / minibatch_size

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per fives)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(ZL), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

In [24]:
def predict(X, parameters):

    L=len(parameters)//2
    params={}
    
    for l in range(1,L):
      params['W'+str(l)] = tf.convert_to_tensor(parameters["W" +str(l)])
      params['b'+str(l)] = tf.convert_to_tensor(parameters["b" +str(l)])
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction

In [25]:
import h5py
import numpy as np
def load_data(file):
  hf = h5py.File(file, 'r')
  classes =hf.get(list(hf.keys())[0])
  x_data=np.array(hf[list(hf.keys())[1]])
  y_data=np.array(hf[list(hf.keys())[2]])
  y_data=y_data.reshape(y_data.shape[0],1).T
  return classes,x_data,y_data

In [26]:
!git clone https://github.com/Jack3690/Rad/

Cloning into 'Rad'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 86 (delta 36), reused 41 (delta 13), pack-reused 0
Unpacking objects: 100% (86/86), done.


In [27]:
classes,X_train_orig,Y_train_orig=load_data('Rad/train_catvnoncat.h5')
_,X_test_orig,Y_test_orig=load_data('Rad/test_catvnoncat.h5')

In [28]:
train_x_flatten=X_train_orig.reshape(X_train_orig.shape[0],-1).T
test_x_flatten=X_test_orig.reshape(X_test_orig.shape[0],-1).T
X_train=train_x_flatten/255
X_test=test_x_flatten/255
Y_train = convert_to_one_hot(Y_train_orig, 2)
Y_test = convert_to_one_hot(Y_test_orig, 2)
print(X_train.shape,Y_train.shape)

(12288, 209) (2, 209)


In [51]:
Layer_dims=[12288,20,7,5,2]

In [ ]:
parameters = model(X_train, Y_train, X_test, Y_test,Layer_dims)

Cost after epoch 0: 0.154007
Cost after epoch 100: 0.073201
Cost after epoch 200: 0.038071
Cost after epoch 300: 0.021238
Cost after epoch 400: 0.012709
Cost after epoch 500: 0.008333
Cost after epoch 600: 0.005724
Cost after epoch 700: 0.003790
Cost after epoch 800: 0.001372
